In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import timedelta, datetime, date
import csv
import pandas
import numpy as np
from itertools import chain

out_path = r'O:\PDI-Barlap\Aoo\HMR_YLD_MNT\EAGLERAYBP_DATA.csv'
#out_path = r'O:\PDI-Barlap\Aoo\HMR_YLD_MNT\FDBHMR6ET_DATA.csv'
product = 'EAGLERAYBP'
#product = 'FDBHMR6ET'
FY_Car = r'O:\PDI-Barlap\Aoo\HMR_YLD_MNT\Calendar.csv'
#start_date = date(2022, 1, 1)
#end_date = date(2022, 3, 14)
start_date = date.today() - timedelta(days=1)
end_date = date.today()

#delta = end_date - start_date
#delta = delta.days

init1 = init1.iloc[0:0]
init2 = init2.iloc[0:0]
init3 = init2.iloc[0:0]

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def findinfo(body,ls):
    for row in body:
        elements = row.find_all('td')
    
        ls_elements_in_row = []
        
        for element in elements:
            text = element.text
            ls_elements_in_row += [text]
            
        ls += [ls_elements_in_row]
    return ls

for single_date in daterange(start_date, end_date):
    date = single_date.strftime("%Y%m%d")
    url = "http://tkfis2.kor.thai.seagate.com:8080/yts/newOP_Binning/op_trend.asp?Type=Time_Date&Trend=True&DSL=&StartDate="+date+"&EndDate="+date+"&RType=MC&BType=&Wxyz=ALL&Shift=ALL&Family=&Product="+product+"&Wafer=&WaferType=&AutoZMVOU=&ReInstate=&Cell=&PILot=&Omit=&Owner=PROD,%20FDB,%20NPD,%20PCTU&Category=&Ratio=&Bin=&EVALFLG=&EVALOPT=EVAL&Eval=&STEPOPT=GStep"        
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "html.parser")
    fil = soup.find("table", {"cellspacing": "2"})
    body = fil.find_all("tr")
    ls = []    
    findinfo(body,ls)
    df = pd.DataFrame(ls[1:2:])
    df.columns = ls[0]
    #init1 = df
    trend = pd.concat([init1,df])
    #trend = trend.iloc[:,[0,1,2,3,4,5]]
    init1 = trend
    #init1 = init1.iloc[0:0]
    

    ttsc1 = df.loc[df.Time_Date == date,'TotalScrapQty'].values[0]
    url1 = "http://tkfis2.kor.thai.seagate.com:8080/yts/newOP_Binning/op_defectgrp.asp?Trend=True&DSL=&StartDate="+date+"&EndDate="+date+"&RType=MC&BType=&Wxyz=ALL&Shift=ALL&Family=&Product="+product+"&Wafer=&WaferType=&Cell=&PILot=&Omit=&Owner=PROD,%20FDB,%20NPD,%20PCTU&RShipQty=0&TotalScrap="+ttsc1+"&CummYield=0&Bin=&EVALFLG=&EVALOPT=EVAL&Eval="
    html_content1 = requests.get(url1).text
    soup1 = BeautifulSoup(html_content1, "html.parser")
    fil1 = soup1.find("table", {"cellspacing": "2"})
    body1 = fil1.find_all("tr")
    ls1 = []
    findinfo(body1,ls1)
    df1 = pd.DataFrame(ls1[1:32:])
    df1.columns = ls1[0]
    df1['Time_Date'] = date
    #init2 = df1
    trend1 = pd.concat([init2,df1])
    init2 = trend1
    #error_code = trend1.loc[trend1.DEF_CODE.isin['110','130','280','290','300']]
    error_code = trend1.loc[trend1.DEF_CODE.isin(['110','130','280','290','300'])]
    #error_code = trend1.loc[trend1.DEF_CODE.isin(['110','130','280','290','300']) & (trend1.REASONQTY != '0')]
    M1 = pd.merge(trend, error_code, on="Time_Date", how="inner")
    M1 = M1.iloc[:,[0,1,2,3,4,5,7,8]]

    #*************************************************************************************************************

    url110 = "http://tkfis2.kor.thai.seagate.com:8080/yts/newOP_Binning/op_defect.asp?DrillDownBy=Defect&DSL=&StartDate="+date+"&EndDate="+date+"&RType=MC&BType=&Wxyz=ALL&Shift=ALL&Family=&Product="+product+"&Wafer=&WaferType=&AutoZMVOU=&ReInstate=&Cell=&PILot=&Omit=&Owner=PROD,%20FDB,%20NPD,%20PCTU&SBRNumber=&EVALFLG=&EVALOPT=EVAL&Eval=&STEPOPT=&Category=&DefectCode=110&Trend=Another&TotalScrap="+ttsc1+"&CummYield=0"
    html_content2 = requests.get(url110).text
    soup2 = BeautifulSoup(html_content2, "html.parser")
    fil2 = soup2.find("table", {"cellspacing": "2"})
    body2 = fil2.find_all("tr")
    body_rows2 = body2[:2]
    ls2 = []
    findinfo(body2,ls2)
    if ls2 == [['Record Not Found']]:
        idx = ["DEF_CODE", "STEPNAME", "REASONCODE", "STARTQTY", "REASONQTY", "BONUSQTY", "DEFECTPERCENTAGE", "NORMALISED", "Drill Down By", "Time_Date"]
        df2 = pd.DataFrame(np.array([["", "", "", "", "", "", "", "", "", date]]), columns=idx)
        e110 = df2
    else:
        df2 = pd.DataFrame(ls2[1::])
        df2.columns = ls2[0]
        df2 = df2.iloc[:-1,:]
        df2['Time_Date'] = date
        e110 = df2    

    url130 = "http://tkfis2.kor.thai.seagate.com:8080/yts/newOP_Binning/op_defect.asp?DrillDownBy=Defect&DSL=&StartDate="+date+"&EndDate="+date+"&RType=MC&BType=&Wxyz=ALL&Shift=ALL&Family=&Product="+product+"&Wafer=&WaferType=&AutoZMVOU=&ReInstate=&Cell=&PILot=&Omit=&Owner=PROD,%20FDB,%20NPD,%20PCTU&SBRNumber=&EVALFLG=&EVALOPT=EVAL&Eval=&STEPOPT=&Category=&DefectCode=130&Trend=Another&TotalScrap="+ttsc1+"&CummYield=0"
    html_content2 = requests.get(url130).text
    soup2 = BeautifulSoup(html_content2, "html.parser")
    fil2 = soup2.find("table", {"cellspacing": "2"})
    body2 = fil2.find_all("tr")
    body_rows2 = body2[:2]
    ls2 = []
    findinfo(body2,ls2)
    if ls2 == [['Record Not Found']]:
        idx = ["DEF_CODE", "STEPNAME", "REASONCODE", "STARTQTY", "REASONQTY", "BONUSQTY", "DEFECTPERCENTAGE", "NORMALISED", "Drill Down By", "Time_Date"]
        df2 = pd.DataFrame(np.array([["", "", "", "", "", "", "", "", "", date]]), columns=idx)
        e130 = df2
    else:
        df2 = pd.DataFrame(ls2[1::])
        df2.columns = ls2[0]
        df2 = df2.iloc[:-1,:]
        df2['Time_Date'] = date
        e130 = df2
        
    url280 = "http://tkfis2.kor.thai.seagate.com:8080/yts/newOP_Binning/op_defect.asp?DrillDownBy=Defect&DSL=&StartDate="+date+"&EndDate="+date+"&RType=MC&BType=&Wxyz=ALL&Shift=ALL&Family=&Product="+product+"&Wafer=&WaferType=&AutoZMVOU=&ReInstate=&Cell=&PILot=&Omit=&Owner=PROD,%20FDB,%20NPD,%20PCTU&SBRNumber=&EVALFLG=&EVALOPT=EVAL&Eval=&STEPOPT=&Category=&DefectCode=280&Trend=Another&TotalScrap="+ttsc1+"&CummYield=0"
    html_content2 = requests.get(url280).text
    soup2 = BeautifulSoup(html_content2, "html.parser")
    fil2 = soup2.find("table", {"cellspacing": "2"})
    body2 = fil2.find_all("tr")
    body_rows2 = body2[:2]
    ls2 = []
    findinfo(body2,ls2)
    if ls2 == [['Record Not Found']]:
        idx = ["DEF_CODE", "STEPNAME", "REASONCODE", "STARTQTY", "REASONQTY", "BONUSQTY", "DEFECTPERCENTAGE", "NORMALISED", "Drill Down By", "Time_Date"]
        df2 = pd.DataFrame(np.array([["", "", "", "", "", "", "", "", "", date]]), columns=idx)
        e280 = df2
    else:
        df2 = pd.DataFrame(ls2[1::])
        df2.columns = ls2[0]
        df2 = df2.iloc[:-1,:]
        df2['Time_Date'] = date
        e280 = df2
        
    url290 = "http://tkfis2.kor.thai.seagate.com:8080/yts/newOP_Binning/op_defect.asp?DrillDownBy=Defect&DSL=&StartDate="+date+"&EndDate="+date+"&RType=MC&BType=&Wxyz=ALL&Shift=ALL&Family=&Product="+product+"&Wafer=&WaferType=&AutoZMVOU=&ReInstate=&Cell=&PILot=&Omit=&Owner=PROD,%20FDB,%20NPD,%20PCTU&SBRNumber=&EVALFLG=&EVALOPT=EVAL&Eval=&STEPOPT=&Category=&DefectCode=290&Trend=Another&TotalScrap="+ttsc1+"&CummYield=0"
    html_content2 = requests.get(url290).text
    soup2 = BeautifulSoup(html_content2, "html.parser")
    fil2 = soup2.find("table", {"cellspacing": "2"})
    body2 = fil2.find_all("tr")
    body_rows2 = body2[:2]
    ls2 = []
    findinfo(body2,ls2)
    if ls2 == [['Record Not Found']]:
        idx = ["DEF_CODE", "STEPNAME", "REASONCODE", "STARTQTY", "REASONQTY", "BONUSQTY", "DEFECTPERCENTAGE", "NORMALISED", "Drill Down By", "Time_Date"]
        df2 = pd.DataFrame(np.array([["", "", "", "", "", "", "", "", "", date]]), columns=idx)
        e290 = df2
    else:
        df2 = pd.DataFrame(ls2[1::])
        df2.columns = ls2[0]
        df2 = df2.iloc[:-1,:]
        df2['Time_Date'] = date
        e290 = df2
        
    url300 = "http://tkfis2.kor.thai.seagate.com:8080/yts/newOP_Binning/op_defect.asp?DrillDownBy=Defect&DSL=&StartDate="+date+"&EndDate="+date+"&RType=MC&BType=&Wxyz=ALL&Shift=ALL&Family=&Product="+product+"&Wafer=&WaferType=&AutoZMVOU=&ReInstate=&Cell=&PILot=&Omit=&Owner=PROD,%20FDB,%20NPD,%20PCTU&SBRNumber=&EVALFLG=&EVALOPT=EVAL&Eval=&STEPOPT=&Category=&DefectCode=300&Trend=Another&TotalScrap="+ttsc1+"&CummYield=0"
    html_content2 = requests.get(url300).text
    soup2 = BeautifulSoup(html_content2, "html.parser")
    fil2 = soup2.find("table", {"cellspacing": "2"})
    body2 = fil2.find_all("tr")
    body_rows2 = body2[:2]
    ls2 = []
    findinfo(body2,ls2)
    if ls2 == [['Record Not Found']]:
        idx = ["DEF_CODE", "STEPNAME", "REASONCODE", "STARTQTY", "REASONQTY", "BONUSQTY", "DEFECTPERCENTAGE", "NORMALISED", "Drill Down By", "Time_Date"]
        df2 = pd.DataFrame(np.array([["", "", "", "", "", "", "", "", "", date]]), columns=idx)
        e300 = df2
    else:
        df2 = pd.DataFrame(ls2[1::])
        df2.columns = ls2[0]
        df2 = df2.iloc[:-1,:]
        df2['Time_Date'] = date
        e300 = df2
        
    MC = pd.concat([e110, e130, e280, e290, e300])
    #init3 = MC
    ML = pd.concat([init3,MC])
    init3 = ML
    ML = ML.iloc[:,[0,2,3,4,6,7,8,9,10]]
    
    #M2 = pd.merge(M1, ML, on=["DEF_CODE", "Time_Date"], how="inner")
    M2 = pd.merge(M1, ML, on=["DEF_CODE", "Time_Date"], how="outer").fillna(0)
    M2['PRODUCT'] = product
    M2 = M2.loc[M2['DESCRIPTION'] != 0]
#M2["Time_Date"] = M2["Time_Date"].astype('string')
#M2.tail(10)
#print(url130)
#error_code
M2

,Time_Date,FinalShipQty,TotalScrapQty,R ShipQty,InProcess Yield,Machine Yield,DEF_CODE,DESCRIPTION,REASONQTY,BONUSQTY,NORMALISED,STEPNAME,REASONCODE,STARTQTY,DEFECTPERCENTAGE,PRODUCT
0,20220315,8238,68,0,99.18,100.00,110,STRIPE HEIGHT,0,0,0,0,0,0,0,EAGLERAYBP
1,20220315,8238,68,0,99.18,100.00,130,BROKEN BAR,0,0,0,0,0,0,0,EAGLERAYBP
2,20220315,8238,68,0,99.18,100.00,280,WIREBOND,0,0,0,0,0,0,0,EAGLERAYBP
3,20220315,8238,68,0,99.18,100.00,290,OSL,0,0,0,0,0,0,0,EAGLERAYBP
4,20220315,8238,68,0,99.18,100.00,300,AKL,0,0,0,0,0,0,0,EAGLERAYBP


In [15]:
#old_data = pd.read_csv(out_path)
#old_data.tail(10)
fiscal = pd.read_csv(FY_Car)
fiscal["Time_Date"] = fiscal["Time_Date"].astype('string')
DATA = pd.merge(fiscal, M2, on="Time_Date", how="inner")
#all = pd.concat([old_data,DATA])
#all.tail(10)
DATA

,WW,Time_Date,FinalShipQty,TotalScrapQty,R ShipQty,InProcess Yield,Machine Yield,DEF_CODE,DESCRIPTION,REASONQTY,BONUSQTY,NORMALISED,STEPNAME,REASONCODE,STARTQTY,DEFECTPERCENTAGE,PRODUCT
0,FY2237,20220315,8238,68,0,99.18,100.00,110,STRIPE HEIGHT,0,0,0,0,0,0,0,EAGLERAYBP
1,FY2237,20220315,8238,68,0,99.18,100.00,130,BROKEN BAR,0,0,0,0,0,0,0,EAGLERAYBP
2,FY2237,20220315,8238,68,0,99.18,100.00,280,WIREBOND,0,0,0,0,0,0,0,EAGLERAYBP
3,FY2237,20220315,8238,68,0,99.18,100.00,290,OSL,0,0,0,0,0,0,0,EAGLERAYBP
4,FY2237,20220315,8238,68,0,99.18,100.00,300,AKL,0,0,0,0,0,0,0,EAGLERAYBP


In [16]:
#DATA.to_csv(out_path, index=None)
#old_data = pd.read_csv(out_path)
#all = pd.concat([old_data,DATA])
#all.tail(10)

In [17]:
#DATA.to_csv(out_path, index=None)
#all.to_csv(out_path, index=None)
#old_data.tail(10)